In [377]:
import pandas as pd
from IPython.display import display, HTML
from glob import glob
import base64

def create_download_link( df, title = "Download CSV file", filename = "data.csv"):
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)



In [371]:
def compute_bytes(d):
    tot = 2.0* d['Floating Point Operations(Half Precision)'] + \
          4.0* d['Floating Point Operations(Single Precision)'] + \
          8.0* d['Floating Point Operations(Double Precision)'] + \
          4.0* d['Integer Instructions']
    tot = tot * d['Invocations']
    return tot

def req_nonlocal_bytes(d):
    tot = (d['Requested Global Load Throughput(bytes/sec)'] + d['Requested Global Store Throughput(bytes/sec)']+ d['Shared Memory Load Throughput(bytes/sec)'] + d['Shared Memory Store Throughput(bytes/sec)']) * (d['Avg. Duration(ns)'] * d['Invocations'])
    #tot = (d['Requested Global Load Throughput(bytes/sec)']) * (d['Avg. Duration(ns)'] * d['Invocations'])
    tot = tot *  1.0E-9 # nanoseconds to seconds
    return tot

def trans_nonlocal_bytes(d):
    tot = (d['Global Load Throughput(bytes/sec)'] + d['Global Store Throughput(bytes/sec)']+ d['Shared Memory Load Throughput(bytes/sec)'] + d['Shared Memory Store Throughput(bytes/sec)']) * d['Avg. Duration(ns)'] * d['Invocations']
    tot = tot* 1.0E-9 # nanoseconds to seconds
    return tot

def req_global_bytes(d):
    tot = (d['Requested Global Load Throughput(bytes/sec)'] + d['Requested Global Store Throughput(bytes/sec)']) * (d['Avg. Duration(ns)'] * d['Invocations'])
    #tot = (d['Requested Global Load Throughput(bytes/sec)']) * (d['Avg. Duration(ns)'] * d['Invocations'])
    tot = tot *  1.0E-9 # nanoseconds to seconds
    return tot

def trans_global_bytes(d):
    tot = (d['Global Load Throughput(bytes/sec)'] + d['Global Store Throughput(bytes/sec)']) * d['Avg. Duration(ns)'] * d['Invocations']
    tot = tot* 1.0E-9 # nanoseconds to seconds
    return tot

def l1_miss_bytes(d):
    tot = (d['Unified Cache Transactions'])* \
           (1.0- (d['Unified Cache Hit Rate(%)']/100.0)) * \
           128.0 *\
           d['Invocations']
    return tot

def l1_access_bytes(d):
    tot = (d['Unified Cache Transactions'])* \
           128.0 *\
           d['Invocations']
    return tot

def dram_bytes(d):
    return (d['Device Memory Read Transactions'] + d['Device Memory Write Transactions']) * 128.0 * d['Invocations']

def unified_hits(d):
    return d['Unified Cache Transactions'] * (d['Unified Cache Hit Rate(%)'] / 100.0) * d['Invocations']

def unified_misses(d):
    return d['Unified Cache Transactions'] * (1.0 - (d['Unified Cache Hit Rate(%)'] / 100.0)) * d['Invocations']
                                              
    

In [372]:
# Ingest Data

data = {}
metrics_filename = 'metrics.csv'
for i in glob('*/'+metrics_filename):
    import re

    # Load data to dataframe, convert to float
    d = pd.read_csv(i, index_col='Name', skipinitialspace=True)
    for col in d.columns:
        d[col] = d[col].astype(float)
    
    # remove the 'memset' pseudo kernel
    if 'memset (0)' in d.index :
        d.drop(['memset (0)'], inplace=True)
    
    # get the name of the dataset from the path
    dset_name = re.sub('/{}$'.format(metrics_filename), '', i)
    data[dset_name] = d 
    



In [373]:
# Print datasets and columns
dsets = pd.DataFrame(sorted(data.keys()))

cols = None
for n, d in data.items():
    c = set(d.columns)
    
    if cols is None:
        cols = set() | c
    else:
        cols = cols | c
    if len(c ^ cols) > 0:
        print("Dataset {} is missing cols {} and has extra cols {}".format(n, cols-c, c-cols))
        print()
columns = pd.DataFrame([sorted(list(cols))])
pd.set_option('display.max_colwidth', -1)
display(HTML(dsets.to_html()))
print((columns.transpose()))


Dataset deeplab_coco_2n_2b_forward is missing cols {'L2 Cache Hit Rate(%)', 'Total number of local load requests from Multiprocessor', 'Total number of local store requests from Multiprocessor', 'Total number of global load requests from Multiprocessor'} and has extra cols set()

Dataset deeplab_coco_2n_2b_backward is missing cols {'L2 Cache Hit Rate(%)', 'Total number of local load requests from Multiprocessor', 'Total number of local store requests from Multiprocessor', 'Total number of global load requests from Multiprocessor'} and has extra cols set()



,0
0,deeplab_coco_2n_2b_backward
1,deeplab_coco_2n_2b_forward
2,gat_sparse_backward_10n_1b
3,gat_sparse_forward_10n_1b
4,gcn_backward_10n_1b
5,gcn_forward_10n_1b
6,graphsage_backward_10n_1b
7,graphsage_forward_10n_1b
8,mpnnv1_backward_10n_1b
9,mpnnv1_forward_10n_1b


                                                           0
0   Avg. Duration(ns)                                       
1   Avg. Dynamic Shared Memory                              
2   Device Memory Read Transactions                         
3   Device Memory Write Transactions                        
4   FP Instructions(Double)                                 
5   FP Instructions(Single)                                 
6   Floating Point Operations(Double Precision)             
7   Floating Point Operations(Half Precision)               
8   Floating Point Operations(Single Precision)             
9   Global Hit Rate in unified l1/tex(%)                    
10  Global Load Throughput(bytes/sec)                       
11  Global Memory Load Efficiency(%)                        
12  Global Memory Store Efficiency(%)                       
13  Global Store Throughput(bytes/sec)                      
14  HP Instructions(Half)                                   
15  Integer Instructions

In [379]:
def gen_stats(d, n, agg=True):
    req_bytes = req_nonlocal_bytes(d)
    trans_bytes = trans_nonlocal_bytes(d)
    req_gl_bytes = req_global_bytes(d)
    trans_gl_bytes = trans_global_bytes(d)
    l1m_bytes = l1_miss_bytes(d)
    l1_bytes = l1_access_bytes(d)
    dram_bytes_m = dram_bytes(d)
    flops = compute_bytes(d)
    u_hits = unified_hits(d)
    u_misses = unified_misses(d)
    
    if agg:
        req_bytes = req_bytes.sum()
        trans_bytes = trans_bytes.sum()
        req_gl_bytes = req_gl_bytes.sum()
        trans_gl_bytes = trans_gl_bytes.sum()
        l1m_bytes = l1m_bytes.sum()
        l1_bytes = l1_bytes.sum()
        dram_bytes_m = dram_bytes_m.sum()
        flops = flops.sum()
        u_hits = u_hits.sum()
        u_misses = u_misses.sum()

    stats = {}
    stats['c_to_m'] = (flops)/(l1_bytes)
    stats['c_to_l1m'] = (flops)/(l1m_bytes)
    stats['c_to_dram'] = (flops)/(dram_bytes_m)
    stats['nonlocal_load_efficiency'] = (req_bytes)/(trans_bytes)
    stats['global_load_efficiency'] = (req_gl_bytes)/(trans_gl_bytes)
    stats['unified_hit_rate'] = (u_hits)/(u_hits + u_misses)
    
    s = pd.Series(stats)
    s.name = n
    return s

res = pd.concat([gen_stats(d,n) for n,d in sorted(data.items())], axis=1).transpose()

#print(res.to_csv())
display(create_download_link(res))
display(HTML(res.to_html()))



,c_to_m,c_to_l1m,c_to_dram,nonlocal_load_efficiency,global_load_efficiency,unified_hit_rate
deeplab_coco_2n_2b_backward,13.830716,32.466585,32.749419,0.826497,0.432150,0.574002
deeplab_coco_2n_2b_forward,4.884669,10.780311,38.828292,0.725249,0.591893,0.546890
gat_sparse_backward_10n_1b,8.692258,15.216129,5.419515,0.909049,0.600083,0.428747
gat_sparse_forward_10n_1b,7.991358,11.208288,3.509798,0.717440,0.511085,0.287013
gcn_backward_10n_1b,4.114401,6.662628,4.204908,0.953458,0.745915,0.382466
gcn_forward_10n_1b,4.230199,7.500044,4.236785,0.945587,0.734459,0.435977
graphsage_backward_10n_1b,13.129392,30.573864,30.437148,0.780647,0.395040,0.570568
graphsage_forward_10n_1b,23.067216,70.316996,45.949826,0.734367,0.276127,0.671954
mpnnv1_backward_10n_1b,11.044919,27.911602,20.642375,0.766640,0.354949,0.604289
mpnnv1_forward_10n_1b,26.490655,48.246340,29.951004,0.970529,0.827548,0.450929


In [380]:
res = pd.concat([gen_stats(d,n) for n,d in sorted(data.items()) if 'deeplab' in n or 'mpnnv1'], axis=1).transpose()

display(create_download_link(res))
display(HTML(res.to_html()))


,c_to_m,c_to_l1m,c_to_dram,nonlocal_load_efficiency,global_load_efficiency,unified_hit_rate
deeplab_coco_2n_2b_backward,13.830716,32.466585,32.749419,0.826497,0.432150,0.574002
deeplab_coco_2n_2b_forward,4.884669,10.780311,38.828292,0.725249,0.591893,0.546890
gat_sparse_backward_10n_1b,8.692258,15.216129,5.419515,0.909049,0.600083,0.428747
gat_sparse_forward_10n_1b,7.991358,11.208288,3.509798,0.717440,0.511085,0.287013
gcn_backward_10n_1b,4.114401,6.662628,4.204908,0.953458,0.745915,0.382466
gcn_forward_10n_1b,4.230199,7.500044,4.236785,0.945587,0.734459,0.435977
graphsage_backward_10n_1b,13.129392,30.573864,30.437148,0.780647,0.395040,0.570568
graphsage_forward_10n_1b,23.067216,70.316996,45.949826,0.734367,0.276127,0.671954
mpnnv1_backward_10n_1b,11.044919,27.911602,20.642375,0.766640,0.354949,0.604289
mpnnv1_forward_10n_1b,26.490655,48.246340,29.951004,0.970529,0.827548,0.450929


In [260]:
from IPython.display import display


from IPython.display import display, HTML
for n, d in data.items():
    s = gen_stats(d, n, agg=False)
    #print(n)
    #display(HTML(s.to_html()))
    
csv = ''
for n, d in data.items():
    s = gen_stats(d, n, agg=False)
    csv += "{}\n".format(n)
    csv += s.to_csv()
    csv += "\n"

print(csv)
display(create_download_link(res))


mpnnv1_forward_10n_1b
Name,c_to_m,nonlocal_load_efficiency,global_load_efficiency,l1_hit_rate
"void kernelPointwiseApply3<TensorMulOp<float>, float, float, float, unsigned int, int=1, int=1, int=1>(OffsetInfo<TensorMulOp<float>, float, unsigned int>, OffsetInfo<float, float, int=1>, OffsetInfo<float, float, int=1>, float, float)",0.6439077814829091,0.998828394210889,0.7952575251131117,0.49418999999999996
"void kernelPointwiseApply2<TensorSigmoidOp<float>, float, float, unsigned int, int=1, int=1>(OffsetInfo<TensorSigmoidOp<float>, float, unsigned int>, OffsetInfo<float, float, int=1>, float, float)",24.265113699389904,0.986863711001642,0.7952575251131117,0.49333000000000005
"void kernelPointwiseApply2<CopyOp<float, unsigned char>, float, unsigned char, unsigned int, int=1, int=1>(OffsetInfo<unsigned char, float, unsigned char>, OffsetInfo<CopyOp<float, unsigned char>, float, unsigned int>, float, float)",0.0,0.9970729111229377,0.7952575251131117,0.57706
"void kernelPointwiseApply2<Tens

In [261]:

from IPython.display import display, HTML
for n, d in data.items():
    s = gen_stats(d, n, agg=False)
    print(n)
    display(HTML(d.to_html()))



mpnnv1_forward_10n_1b


,Invocations,Avg. Duration(ns),Registers/Thread,Static Shared Memory,Avg. Dynamic Shared Memory,Global Memory Store Efficiency(%),L2 Cache Hit Rate(%),HP Instructions(Half),Unified Cache Transactions,Floating Point Operations(Double Precision),Floating Point Operations(Single Precision),Total number of global load requests from Multiprocessor,Total number of local load requests from Multiprocessor,Total number of local store requests from Multiprocessor,Shared Memory Load Throughput(bytes/sec),Floating Point Operations(Half Precision),Shared Memory Store Throughput(bytes/sec),Global Memory Load Efficiency(%),Global Hit Rate in unified l1/tex(%),L2 Transactions (Texture Reads),Local Hit Rate(%),Requested Global Load Throughput(bytes/sec),Requested Global Store Throughput(bytes/sec),Global Load Throughput(bytes/sec),L2 Transactions (Texture Writes),Global Store Throughput(bytes/sec),Unified Cache Hit Rate(%),Shared Memory Efficiency(%),FP Instructions(Single),FP Instructions(Double),Integer Instructions,Shared Store Transactions,Shared Load Transactions
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"void kernelPointwiseApply3<TensorMulOp<float>, float, float, float, unsigned int, int=1, int=1, int=1>(OffsetInfo<TensorMulOp<float>, float, unsigned int>, OffsetInfo<float, float, int=1>, OffsetInfo<float, float, int=1>, float, float)",23.0,1132.0,11.0,0.0,0.0,98.852,0.000,0.0,661.0,0.0,2.641000e+03,661.0,0.0,0.0,0.000000e+00,0.0,0.000000e+00,98.852,0.000,661.0,0.0,1.449300e+10,7.246000e+09,1.451000e+10,330.0,7.255000e+09,49.419,0.000,2.641000e+03,0.0,69506.0,0.0,0.0
"void kernelPointwiseApply2<TensorSigmoidOp<float>, float, float, unsigned int, int=1, int=1>(OffsetInfo<TensorSigmoidOp<float>, float, unsigned int>, OffsetInfo<float, float, int=1>, float, float)",10.0,1152.0,14.0,0.0,0.0,98.684,0.000,0.0,38.0,0.0,4.200000e+03,38.0,0.0,0.0,0.000000e+00,0.0,0.000000e+00,98.684,0.000,38.0,0.0,6.010000e+08,6.010000e+08,6.090000e+08,38.0,6.090000e+08,49.333,0.000,5.100000e+03,0.0,7748.0,0.0,0.0
"void kernelPointwiseApply2<CopyOp<float, unsigned char>, float, unsigned char, unsigned int, int=1, int=1>(OffsetInfo<unsigned char, float, unsigned char>, OffsetInfo<CopyOp<float, unsigned char>, float, unsigned int>, float, float)",13.0,1255.0,10.0,0.0,0.0,98.980,0.000,0.0,555.0,0.0,0.000000e+00,555.0,0.0,0.0,0.000000e+00,0.0,0.000000e+00,95.165,0.000,139.0,0.0,3.747000e+09,1.499100e+10,3.758000e+09,555.0,1.500100e+10,57.706,0.000,0.000000e+00,0.0,85224.0,0.0,0.0
"void kernelPointwiseApply2<TensorGTValueOp<float, unsigned char>, unsigned char, float, unsigned int, int=1, int=2>(OffsetInfo<unsigned char, unsigned char, float>, OffsetInfo<TensorGTValueOp<float, unsigned char>, unsigned char, unsigned int>, unsigned char, float)",13.0,1272.0,8.0,0.0,0.0,95.165,69.613,0.0,555.0,0.0,0.000000e+00,555.0,0.0,0.0,0.000000e+00,0.0,0.000000e+00,33.749,0.000,145.0,0.0,6.550000e+08,3.556000e+09,3.712000e+09,139.0,3.566000e+09,73.902,0.000,4.442000e+03,0.0,142974.0,0.0,0.0
"void kernelReduceContigDim<float, unsigned int, float, thrust::identity<float>, ReduceAdd<float>, thrust::identity<float>, int=1, int=1>(TensorInfo<float, unsigned int>, TensorInfo<float, unsigned int>, unsigned int, unsigned int, float, float, thrust::identity<float>, float)",13.0,1915.0,15.0,0.0,324.0,12.500,33.523,0.0,307.0,0.0,5.769000e+03,307.0,0.0,0.0,7.264900e+10,0.0,6.713000e+09,74.769,0.000,370.0,0.0,3.223000e+09,1.380000e+08,4.525000e+09,76.0,1.111000e+09,43.025,27.039,7.615000e+03,0.0,122538.0,134.0,1269.0
"void CatArrayBatchedCopy<float, unsigned int, int=2>(float*, CatArrInputTensor<float, unsigned int>*, OutputTensorSizeStride<unsigned int, unsigned int=4>, int, unsigned int)",10.0,2038.0,16.0,0.0,0.0,64.243,73.682,0.0,7154.0,0.0,0.000000e+00,6238.0,0.0,0.0,0.000000e+00,0.0,0.000000e+00,33.377,0.000,1997.0,0.0,6.788000e+09,4.648000e+09,2.032100e+10,710.0,7.235000e+09,66.109,0.000,3.650000e+03,0.0,716414.0,0.0,0.0
"void kernelReduceNoncontigDim_shared<float, unsigned int, float, thrus

gcn_inference


,Total Duration (ms),Kernel Time/GPU Time,FOPs(HP)/Bytes Read+Write,FOPs(SP)/Bytes Read+Write,FOPs(DP)/Bytes Read+Write,Global Store Req Throughput/Global Store Actual Throughput,Global Load Req Throughput/Global Load Actual Throughput,Invocations,Avg. Duration(ns),Registers/Thread,Static Shared Memory,Avg. Dynamic Shared Memory,Global Memory Store Efficiency(%),Unified Cache Transactions,HP Instructions(Half),Floating Point Operations(Double Precision),Floating Point Operations(Single Precision),Global Hit Rate in unified l1/tex(%),Local Hit Rate(%),L2 Transactions (Texture Reads),Requested Global Load Throughput(bytes/sec),Requested Global Store Throughput(bytes/sec),L2 Transactions (Texture Writes),Global Load Throughput(bytes/sec),Global Store Throughput(bytes/sec),Unified Cache Hit Rate(%),Shared Memory Efficiency(%),FP Instructions(Single),FP Instructions(Double),Integer Instructions,Shared Store Transactions,Shared Load Transactions,Floating Point Operations(Half Precision),Shared Memory Load Throughput(bytes/sec),Shared Memory Store Throughput(bytes/sec),Global Memory Load Efficiency(%)
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"void kernelTransformReduceInnermostDimIndex<long, long, MaxValuePair<long, long>>(long*, long*, long*, unsigned int, unsigned int, thrust::pair<long, long>, long)",7886.49588,0.450180,0.0,0.000000,0.000000,NaN,0.888785,20.0,394226570.0,16.0,8704.0,0.0,50.000,11723404.0,0.0,0.0,0.000000e+00,25.000,0.0,13188828.0,9.510000e+08,0.000000e+00,2.0,1.070000e+09,0.000000e+00,25.000,68.636,0.000000e+00,0.0,4.689460e+08,80.0,30.0,0.0,0.000000e+00,0.000000e+00,88.889
"void kernelTransformReduceInnermostDimIndex<float, long, MaxValuePair<float, long>>(float*, long*, float*, unsigned int, unsigned int, thrust::pair<float, long>, float)",7884.53140,0.450068,0.0,0.000000,0.000000,0.374981,0.661290,10.0,143001.0,18.0,6528.0,0.0,37.500,1048344.0,0.0,0.0,0.000000e+00,39.806,0.0,1354110.0,2.000000e+11,1.466100e+10,174724.0,3.030000e+11,3.909800e+10,41.177,29.063,1.956909e+07,0.0,9.121202e+07,1397848.0,2096688.0,0.0,1.880000e+12,1.250000e+12,66.129
"void thrust::cuda_cub::core::_kernel_agent<thrust::cuda_cub::__merge_sort::BlockSortAgent<thrust::device_ptr<long>, thrust::device_ptr<long>, long, ThrustLTOp<long>, thrust::detail::integral_constant<bool, bool=1>, thrust::detail::integral_constant<bool, bool=0>>, bool, thrust::device_ptr<long>, thrust::device_ptr<long>, long, long*, long*, ThrustLTOp<long>>(thrust::device_ptr<long>, thrust::device_ptr<long>, long, long, ThrustLTOp<long>, bool, bool=1)",504.64624,0.028806,0.0,0.000000,0.000000,1.000000,0.994184,20.0,6193701.0,88.0,0.0,16912.0,100.000,12456204.0,0.0,0.0,0.000000e+00,2.273,0.0,11815002.0,6.068700e+10,6.056800e+10,11723402.0,6.104200e+10,6.056800e+10,1.154,25.105,0.000000e+00,0.0,4.155477e+09,102605940.0,196581671.0,0.0,4.060000e+12,2.120000e+12,99.419
sgemm_32x32x32_NN,388.55268,0.022180,0.0,0.004116,0.000000,0.802817,0.214527,20.0,1004137.0,153.0,16912.0,0.0,87.273,14051290.0,0.0,0.0,5.204575e+09,82.336,0.0,12979354.0,4.430000e+11,2.644800e+10,1033782.0,2.070000e+12,3.294400e+10,76.370,144.834,2.688262e+09,0.0,3.345765e+08,7863480.0,13979520.0,0.0,1.780000e+12,1.000000e+12,21.917
"void convert_CooToCsr_kernel<int=0>(int const *, int, int, int*)",189.10506,0.010795,0.0,0.000000,0.000000,0.139147,0.820514,20.0,288362.0,12.0,0.0,512.0,13.917,2953931.0,0.0,0.0,0.000000e+00,48.684,0.0,3600101.0,3.280000e+11,3.231000e+09,209253.0,4.000000e+11,2.322000e+10,47.402,99.479,6.144000e+04,0.0,4.291273e+08,738482.0,1476964.0,0.0,6.560000e+11,3.280000e+11,82.051
"void kernelTransformReduceInnermostDimIndex<long, long, MinValuePair<long, long>>(long*, long*, long*, unsigned int, unsigned int, thrust::pair<long, long>, long)",123.87402,0.007071,0.0,0.000000,0.000000,NaN,0.888681,20.0,394324794.0,16.0,8704.0,0.0,50.000,11723404.0,0.0,0.0,0.000000e+00,25.000,0.0,13188828.0,9.500000e+08,0.000000e+00,2.0,1.069000e+09,0.000000e+00,25.000,68.636,0.000000e+00,0.0,4.689460e+08,80

gat_sparse_backward_10n_1b


...

,Invocations,Avg. Duration(ns),Registers/Thread,Static Shared Memory,Avg. Dynamic Shared Memory,Global Memory Store Efficiency(%),L2 Cache Hit Rate(%),HP Instructions(Half),Unified Cache Transactions,Floating Point Operations(Double Precision),Floating Point Operations(Single Precision),Total number of global load requests from Multiprocessor,Total number of local load requests from Multiprocessor,Total number of local store requests from Multiprocessor,Shared Memory Load Throughput(bytes/sec),Floating Point Operations(Half Precision),Shared Memory Store Throughput(bytes/sec),Global Memory Load Efficiency(%),Global Hit Rate in unified l1/tex(%),L2 Transactions (Texture Reads),Local Hit Rate(%),Requested Global Load Throughput(bytes/sec),Requested Global Store Throughput(bytes/sec),Global Load Throughput(bytes/sec),L2 Transactions (Texture Writes),Global Store Throughput(bytes/sec),Unified Cache Hit Rate(%),Shared Memory Efficiency(%),FP Instructions(Single),FP Instructions(Double),Integer Instructions,Shared Store Transactions,Shared Load Transactions
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"void scalePackedTensor_kernel<float, float>(cudnnTensor4dStruct, float*, float)",250.0,849.0,8.0,0.0,0.0,99.090,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.000000e+00,0.000,0.000,0.0,0.0,0.000000e+00,2.276000e+09,0.000000e+00,112.0,2.278000e+09,49.666,0.000,896.0,0.0,14935.0,0.0,0.0
"void kernelPointwiseApply2<TensorAddOp<float>, float, float, unsigned int, int=1, int=1>(OffsetInfo<TensorAddOp<float>, float, unsigned int>, OffsetInfo<float, float, int=1>, float, float)",740.0,1009.0,10.0,0.0,0.0,81.280,33.333,0.0,79.0,0.0,312.0,79.0,0.0,0.0,0.000000e+00,0.0,0.000000e+00,81.280,0.000,79.0,0.0,1.519000e+09,7.590000e+08,1.534000e+09,39.0,7.660000e+08,40.563,0.000,312.0,0.0,7656.0,0.0,0.0
cudnn::maxwell::gemm::computeBOffsetsKernel(cudnn::maxwell::gemm::ComputeBOffsetsParams),480.0,1044.0,12.0,0.0,0.0,43.490,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.000000e+00,0.000,0.000,0.0,0.0,0.000000e+00,2.800000e+07,0.000000e+00,3.0,6.400000e+07,11.181,0.000,0.0,0.0,1337.0,0.0,0.0
"void kernelPointwiseApply1<TensorFillOp<float>, float, unsigned int, int=1>(OffsetInfo<TensorFillOp<float>, float, unsigned int>, float, float)",10.0,1161.0,8.0,0.0,0.0,12.500,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.000000e+00,0.000,0.000,0.0,0.0,0.000000e+00,2.000000e+06,0.000000e+00,1.0,2.000000e+07,0.000,0.000,0.0,0.0,2058.0,0.0,0.0
"void kernelPointwiseApply3<TensorAddOp<float>, float, float, float, unsigned int, int=1, int=1, int=1>(OffsetInfo<TensorAddOp<float>, float, unsigned int>, OffsetInfo<float, float, int=1>, OffsetInfo<float, float, int=1>, float, float)",590.0,1293.0,11.0,0.0,0.0,99.622,0.000,0.0,3391.0,0.0,13561.0,3391.0,0.0,0.0,0.000000e+00,0.0,0.000000e+00,99.622,0.000,3390.0,0.0,4.624200e+10,2.312100e+10,4.624900e+10,1695.0,2.312400e+10,49.809,0.000,13561.0,0.0,353841.0,0.0,0.0
"void kernelPointwiseApply2<CopyOp<long, long>, long, long, unsigned int, int=1, int=1>(OffsetInfo<long, long, long>, OffsetInfo<CopyOp<long, long>, long, unsigned int>, long, long)",10.0,1315.0,8.0,0.0,0.0,95.068,0.000,0.0,98.0,0.0,0.0,49.0,0.0,0.0,0.000000e+00,0.0,0.000000e+00,48.939,0.000,194.0,0.0,1.582000e+09,1.582000e+09,3.170000e+09,97.0,1.589000e+09,0.000,0.000,0.0,0.0,9286.0,0.0,0.0
cudnn::maxwell::gemm::computeOffsetsKernel(cudnn::maxwell::gemm::ComputeOffsetsParams),230.0,1317.0,16.0,0.0,0.0,67.150,61.143,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.000000e+00,0.000,0.000,0.0,0.0,0.000000e+00,1.344000e+09,0.000000e+00,85.0,1.573000e+09,44.581,0.000,0.0,0.0,28972.0,0.0,0.0
"void kernelPointwiseApply3<ThresholdUpdateGradInput<float>, float, float, float, unsigned int, int=1, int=1, int=1>(OffsetInfo<ThresholdUpdateGradInput<float>, float, unsigned int>, OffsetInfo<float, float, int=1>, OffsetInfo<float, float, int=1>, float, float)",120.0,1342.0,10.0,0.0,0.0,98.887,0.000,0.0,952.0,0.0,0.0,952.0,0.0,0.0,0.000000e+00,0.0,0.000000e+00,76.862,0.00

gcn_training


,Invocations,Avg. Duration(ns),Registers/Thread,Static Shared Memory,Avg. Dynamic Shared Memory,Global Memory Store Efficiency(%),HP Instructions(Half),Unified Cache Transactions,Floating Point Operations(Double Precision),Floating Point Operations(Single Precision),Global Hit Rate in unified l1/tex(%),Local Hit Rate(%),L2 Transactions (Texture Reads),Requested Global Load Throughput(bytes/sec),Requested Global Store Throughput(bytes/sec),Global Load Throughput(bytes/sec),L2 Transactions (Texture Writes),Global Store Throughput(bytes/sec),Unified Cache Hit Rate(%),Shared Memory Efficiency(%),FP Instructions(Single),FP Instructions(Double),Integer Instructions,Shared Store Transactions,Shared Load Transactions,Floating Point Operations(Half Precision),Shared Memory Load Throughput(bytes/sec),Shared Memory Store Throughput(bytes/sec),Global Memory Load Efficiency(%)
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"void kernelPointwiseApply1<TensorFillOp<float>, float, unsigned int, int=1>(OffsetInfo<TensorFillOp<float>, float, unsigned int>, float, float)",10.0,1672.0,8.0,0.0,0.0,12.500,0.0,0.0,0.0,0.000000e+00,0.000,0.000,0.0,0.000000e+00,2.000000e+06,0.000000e+00,1.0,1.800000e+07,0.000,0.000,0.000000e+00,0.0,2.058000e+03,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000
"void kernelPointwiseApply1<TensorFillOp<long>, long, unsigned int, int=1>(OffsetInfo<TensorFillOp<long>, long, unsigned int>, long, long)",20.0,1685.0,10.0,0.0,0.0,25.000,0.0,0.0,0.0,0.000000e+00,0.000,0.000,0.0,0.000000e+00,4.000000e+06,0.000000e+00,1.0,1.800000e+07,0.000,0.000,0.000000e+00,0.0,2.058000e+03,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000
"void thrust::cuda_cub::core::_kernel_agent<thrust::cuda_cub::__parallel_for::ParallelForAgent<thrust::cuda_cub::__tabulate::functor<thrust::device_ptr<long>, LinspaceOp<long, long>, long>, long>, thrust::cuda_cub::__tabulate::functor<thrust::device_ptr<long>, LinspaceOp<long, long>, long>, long>(thrust::device_ptr<long>, long)",10.0,1699.0,19.0,0.0,0.0,93.182,0.0,0.0,0.0,0.000000e+00,0.000,0.000,0.0,0.000000e+00,1.920000e+08,0.000000e+00,11.0,2.060000e+08,0.000,0.000,0.000000e+00,0.0,4.281000e+03,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000
"void scal_kernel<float, float, int=1, bool=1, int=6, int=5, int=5, int=3>(cublasTransposeParams<float>, float const *, float*, float const *)",10.0,1882.0,18.0,0.0,0.0,85.417,0.0,0.0,0.0,0.000000e+00,0.000,0.000,0.0,0.000000e+00,2.787000e+09,0.000000e+00,192.0,3.264000e+09,45.455,0.000,0.000000e+00,0.0,1.996800e+04,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000
_ZN2at6native18elementwise_kernelILi512ELi1EZNS0_16gpu_unary_kernelIZNS0_17gpu_binary_kernelIZNS0_15add_kernel_implIfEEvRNS_14TensorIteratorEN3c106ScalarEEUlffE_EEvS6_RKT_EUlfE0_EEvS6_SC_EUliE_EEviT1_,40.0,2049.0,8.0,0.0,0.0,96.354,0.0,323.0,0.0,5.172000e+03,48.864,0.000,323.0,5.047000e+09,5.047000e+09,5.051000e+09,323.0,5.051000e+09,48.864,0.000,2.586000e+03,0.0,3.692300e+04,0.0,0.0,0.0,0.000000e+00,0.000000e+00,96.354
_ZN2at6native18elementwise_kernelILi512ELi1EZNS0_16gpu_unary_kernelIZNS0_17gpu_binary_kernelIZNS0_15mul_kernel_implIfEEvRNS_14TensorIteratorEEUlffE_EEvS6_RKT_EUlfE0_EEvS6_SA_EUliE_EEviT1_,80.0,2052.0,8.0,0.0,0.0,96.354,0.0,323.0,0.0,2.586000e+03,48.864,0.000,323.0,5.038000e+09,5.038000e+09,5.042000e+09,323.0,5.042000e+09,48.864,0.000,2.586000e+03,0.0,3.692300e+04,0.0,0.0,0.0,0.000000e+00,0.000000e+00,96.354
"void kernelPointwiseApply3<TensorAddCMulOp<float>, float, float, float, unsigned int, int=1, int=1, int=1>(OffsetInfo<TensorAddCMulOp<float>, float, unsigned int>, OffsetInfo<float, float, int=1>, OffsetInfo<float, float, int=1>, float, float)",40.0,2080.0,12.0,0.0,0.0,96.354,0.0,970.0,0.0,7.758000e+03,48.864,0.000,970.0,1.491800e+10,4.972000e+09,1.492800e+10,323.0,4.975000e+09,48.864,0.000,5.172000e+03,0.0,6.867300e+04,0.0,0.0,0.0,0.000000e+00,0.000000e+00,96.354
_ZN2at6native18elementwise_kernelILi512ELi1EZNS0_17gpu_binary_kernelIZNS0_15add_kernel_implIfEEvRNS_14TensorIteratorEN3c106ScalarEEUlffE_EEvS5_RKT_EUliE_EEviT1_

multiscale_forward_10n_10b


,Invocations,Avg. Duration(ns),Registers/Thread,Static Shared Memory,Avg. Dynamic Shared Memory,Global Memory Store Efficiency(%),L2 Cache Hit Rate(%),HP Instructions(Half),Unified Cache Transactions,Floating Point Operations(Double Precision),Floating Point Operations(Single Precision),Total number of global load requests from Multiprocessor,Total number of local load requests from Multiprocessor,Total number of local store requests from Multiprocessor,Shared Memory Load Throughput(bytes/sec),Floating Point Operations(Half Precision),Shared Memory Store Throughput(bytes/sec),Global Memory Load Efficiency(%),Global Hit Rate in unified l1/tex(%),L2 Transactions (Texture Reads),Local Hit Rate(%),Requested Global Load Throughput(bytes/sec),Requested Global Store Throughput(bytes/sec),Global Load Throughput(bytes/sec),L2 Transactions (Texture Writes),Global Store Throughput(bytes/sec),Unified Cache Hit Rate(%),Shared Memory Efficiency(%),FP Instructions(Single),FP Instructions(Double),Integer Instructions,Shared Store Transactions,Shared Load Transactions
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"void kernelPointwiseApply2<TensorMaxValueOp<float>, float, float, unsigned int, int=1, int=1>(OffsetInfo<TensorMaxValueOp<float>, float, unsigned int>, OffsetInfo<float, float, int=1>, float, float)",120.0,998.0,8.0,0.0,0.0,98.887,0.000,0.0,486.0,0.0,0.0,486.0,0.0,0.0,0.000000e+00,0.0,0.000000e+00,98.887,0.000,485.0,0.0,8.150000e+09,8.150000e+09,8.157000e+09,485.0,8.157000e+09,49.654,0.000,3884.0,0.0,78937.0,0.0,0.0
"void kernelPointwiseApply1<TensorAddConstantOp<long>, long, unsigned int, int=1>(OffsetInfo<TensorAddConstantOp<long>, long, unsigned int>, long, long)",120.0,1172.0,8.0,0.0,0.0,25.000,50.000,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.000000e+00,0.0,0.000000e+00,25.000,0.000,1.0,0.0,4.000000e+06,4.000000e+06,1.700000e+07,1.0,1.700000e+07,0.000,0.000,0.0,0.0,2060.0,0.0,0.0
cudnn::maxwell::gemm::computeOffsetsKernel(cudnn::maxwell::gemm::ComputeOffsetsParams),250.0,1244.0,16.0,0.0,0.0,67.150,61.302,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.000000e+00,0.000,0.000,0.0,0.0,0.000000e+00,1.330000e+09,0.000000e+00,85.0,1.553000e+09,44.581,0.000,0.0,0.0,28972.0,0.0,0.0
"void add_tensor_kernel_v3<int=2, float, float, int=128, int=1, int=1, int=4, int=2>(cudnnTensorStruct, float*, cudnnTensorStruct, float const *, float, float)",250.0,1822.0,28.0,0.0,0.0,87.283,65.169,0.0,719.0,0.0,9459.0,719.0,0.0,0.0,0.000000e+00,0.0,0.000000e+00,61.609,0.000,539.0,0.0,5.066000e+09,4.990000e+09,5.696000e+09,481.0,5.086000e+09,57.848,0.000,8225.0,0.0,103312.0,0.0,0.0
"void gemv2N_kernel_val<float, float, float, int=128, int=32, int=4, int=4, int=1>(float, float, cublasGemv2Params_v2<float, float, float>)",25.0,2266.0,32.0,2560.0,0.0,46.972,65.419,0.0,1885.0,0.0,26357.0,1885.0,0.0,0.0,7.099100e+10,0.0,2.783700e+10,42.017,0.000,4065.0,0.0,1.487700e+10,7.200000e+07,3.882100e+10,16.0,1.490000e+08,17.717,22.289,16460.0,0.0,205225.0,734.0,1870.0
"void CatArrayBatchedCopy<float, unsigned int, int=4>(float*, CatArrInputTensor<float, unsigned int>*, OutputTensorSizeStride<unsigned int, unsigned int=4>, int, unsigned int)",120.0,2379.0,24.0,0.0,0.0,91.404,74.517,0.0,9698.0,0.0,0.0,7767.0,0.0,0.0,0.000000e+00,0.0,0.000000e+00,31.433,0.000,2671.0,0.0,1.267500e+10,9.996000e+09,2.645600e+10,1065.0,1.096500e+10,69.730,0.000,23139.0,0.0,1412678.0,0.0,0.0
"void cudnn::detail::bn_fw_tr_1C11_singleread<float, int=512, bool=1, int=1, int=2, int=0>(cudnnTensorStruct, float const *, cudnn::detail::bn_fw_tr_1C11_singleread<float, int=512, bool=1, int=1, int=2, int=0>, cudnnTensorStruct*, float const *, float const , float, float, float*, float const *, float const *, float const *, float, float, cudnn::reduced_divisor, int, float*, cudnn::detail::bnFwPersistentState*, int, float, float, float, int, float, float, cudnnStatus_t*, bool)",120.0,4152.0,36.0,144.0,3072.0,57.345,55.770,0.0,3584.0,0.0,227288.0,3584.0,0.0,0.0,3.087900e+10,0.0,2.604400e+10,36.948,56.181,1193.0,0.0,8.476000e+09,7

KeyError: 'gat_dense_forward_10n_1b/metrics.csv'